In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from pycaret.regression import *

In [31]:
#LABEL_ENCODER = None
'''

def remove_null(data):
    data.dropna(inplace=True)
    return data

def convert_dates_to_one_format(data):
    data['Travel Date'] = pd.to_datetime(data['Travel Date'], errors='coerce')
    data['Travel Date'] = data['Travel Date'].dt.strftime('%Y-%m-%d')
    return data

def replace_space_with_underscore(name):
    return name.replace(' ', '_')

def create_label_encoding(data):
    global LABEL_ENCODER
    cat_cols = data.select_dtypes(include=['object']).columns
    LABEL_ENCODER = LabelEncoder()
    for col in cat_cols:
        data[col] = LABEL_ENCODER.fit_transform(data[col])
        np.save('classes_{0}.npy'.format(replace_space_with_underscore(col)), LABEL_ENCODER.classes_, allow_pickle=True)

def change_int32_to_int64(data):
    for col in data.columns:
        if data[col].dtype == 'int64':
            data[col] = data[col].astype('int32')

def data_preprocessing(data):
    data.drop(['Uniq Id'], axis=1, inplace=True)
    remove_null(data)
    change_int32_to_int64(data)
    convert_dates_to_one_format(data)
    create_label_encoding(data)

'''
### preprocessing 

def split_category(value):
    vals = []
    if '|' in value:
        vals = value.split('|')
    else:
        vals.append(value)
    return vals

def split_columns_with_multiple_values(data):
    data['Airline'] = data['Airline'].apply(split_category)
    #data['Destination'] = data['Destination'].apply(split_category)
    data['Places Covered'] = data['Places Covered'].apply(split_category)
    data['Sightseeing Places Covered'] = data['Sightseeing Places Covered'].apply(split_category)
    return data

def one_hot_encoding_on_columns(data):
    dummy_type = pd.get_dummies(data['Package Type'], prefix='type')
    data.drop(columns=['Package Type'], inplace=True)
    data = pd.concat([data, dummy_type], axis=1)

    dummy_city = pd.get_dummies(data['Start City'], prefix='sc')
    data.drop(columns=['Start City'], inplace=True)
    data = pd.concat([data, dummy_city], axis=1)
    print("INSIDE ONE HOT ENCODER")
    
    '''
    label_encoder2 = LabelEncoder().fit(data['Cancellation Rules'])
    data['Cancellation Rules'] = label_encoder2.transform(data['Cancellation Rules'])
    return data
    '''
    return data

def show_category(series):
    values = {}
    for val in series:
        for each in val:
            if each in values:
                values[each] += 1
            else:
                values[each] = 1
    return values

def make_feature_col(series, all_keys):
    feature_dict = {}
    for key in all_keys:
        feature_dict[key] = []

    for items in series:
        for key in all_keys:
            if key not in items:
                feature_dict[key].append(0)
            else:
                feature_dict[key].append(1)

    return pd.DataFrame(feature_dict)

def structuring_columns(data):
    change = lambda pc: [each+'_ae' for each in pc]
    print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    display(data)
    A_all_keys = show_category(data['Airline']).keys()
    A_all_keys = change(A_all_keys)
    airline = make_feature_col(data['Airline'], A_all_keys)
    data = pd.concat([data, airline], axis=1)
    '''
    change = lambda pc: [each+'_ds' for each in pc]
    D_all_keys = show_category(data['Destination']).keys()
    D_all_keys = change(D_all_keys)
    destination = make_feature_col(data['Destination'], D_all_keys)
    data = pd.concat([data, destination], axis=1)
    '''
    change = lambda pc: [each+'_pc' for each in pc]
    PC_all_keys = show_category(data['Places Covered']).keys()
    PC_all_keys = change(PC_all_keys)
    p_covered = make_feature_col(data['Places Covered'], PC_all_keys)
    data = pd.concat([data, p_covered], axis=1)

    return data

def remove_useless_columns(data):
    data.drop('Uniq Id', inplace=True, axis=1)
    data.drop('Package Name', inplace=True, axis=1)
    data.drop('Destination', inplace=True, axis=1)
    data.drop('Cancellation Rules', inplace=True, axis=1)
    return data
  
def preprocess_data(data):
    data = remove_useless_columns(data)
    display(data)
    data = split_columns_with_multiple_values(data)
    display(data)
    data = one_hot_encoding_on_columns(data)
    display(data)
    data = structuring_columns(data)
    return(data)
    
 ### splitting data 
'''
def split_data(data):
    features = ['Flight Stops', 'Meals'] #deleted cancellation rules
    other_features = data.columns[13:].to_list()
    features.extend(other_features)
    target = ['Per Person Price']

    X = data[features]
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    print('Shape of train set:', X_train.shape)
    print('Shape of test set:', X_test.shape)

    return X_train, X_test, y_train, y_test
'''

"\ndef split_data(data):\n    features = ['Flight Stops', 'Meals'] #deleted cancellation rules\n    other_features = data.columns[13:].to_list()\n    features.extend(other_features)\n    target = ['Per Person Price']\n\n    X = data[features]\n    y = data[target]\n\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)\n\n    print('Shape of train set:', X_train.shape)\n    print('Shape of test set:', X_test.shape)\n\n    return X_train, X_test, y_train, y_test\n"

In [32]:
train = pd.read_csv('dataset\Train.csv')
validation = pd.read_csv('dataset\Test.csv')

final_train_data = preprocess_data(train)
final_validation_data = preprocess_data(validation)

,Package Type,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Per Person Price
0,Standard,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,30-07-2021,Not Available,Mumbai,Not Available,2,3,Not Available,11509.0
1,Premium,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,08-12-2021,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,22485.5
2,Luxury,2N Udaipur . 1N Chittorgarh,Udaipur|Chittorgarh,26-04-2021,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,12421.5
3,Premium,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Munnar|Kumarakom|Allepey|Kovalam and Poovar,27-08-2021,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,IndiGo,0,5,Mattupetty Dam | Echo Point | Tata Tea Museum...,35967.0
4,Premium,4N Pattaya . 3N Bangkok,Pattaya|Bangkok,12-12-2021,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,Spicejet|Go Air,0,5,"Coral Island Tour with Indian Lunch, Join Spe...",25584.0
...,...,...,...,...,...,...,...,...,...,...,...
20995,Deluxe,2N Shimla . 3N Manali . 1N New Delhi,Shimla|Manali|New Delhi,15-04-2021,Summit Thistle Villa Resort & Spa:Four|Snow Va...,Mumbai,Not Available,1,3,Mall road | Indira Bunglow | Himalayan Zoo | ...,16168.5
20996,Luxury,1N Srinagar . 1N Gulmarg . 2N Pahalgam . 3N Sr...,Srinagar|Gulmarg|Pahalgam|Srinagar,30-08-2021,The Orchard Retreat & Spa:4.6|The Khyber Himal...,New Delhi,Vistara|Vistara,0,4,Dal Lake | Gondola Point | Avantipura Ruins |...,33770.5
20997,Deluxe,5N Colombo,Colombo,27-02-2021,Mount Lavinia Hotel-MMT HOLIDAYS SPECIAL:4.0,New Delhi,IndiGo|IndiGo|IndiGo|IndiGo,1,3,Sigiriya Excursion | Nuwara Eliya Excursion,29964.0
20998,Deluxe,2N Darjeeling . 2N Pelling . 2N Gangtok,Darjeeling|Pelling|Gangtok,13-01-2022,Ramada by Wyndham Darjeeling Gandhi Road:4.4|T...,New Delhi,IndiGo,1,3,Tiger Hill | Himalayan Mountaineering Institu...,28145.5


,Package Type,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Per Person Price
0,Standard,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,"[New Delhi, Shimla, Manali, Chandigarh]",30-07-2021,Not Available,Mumbai,[Not Available],2,3,[Not Available],11509.0
1,Premium,1N Srinagar . 2N Pahalgam . 1N Srinagar,"[Srinagar, Pahalgam, Srinagar]",08-12-2021,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,"[IndiGo, IndiGo]",0,5,"[ Dal Lake , Avantipura Ruins , Mughal Garde...",22485.5
2,Luxury,2N Udaipur . 1N Chittorgarh,"[Udaipur, Chittorgarh]",26-04-2021,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,[IndiGo],0,4,"[ Lake Pichola , Jag Mandir Palace , Saheliy...",12421.5
3,Premium,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,"[Munnar, Kumarakom, Allepey, Kovalam and Poovar]",27-08-2021,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,[IndiGo],0,5,"[ Mattupetty Dam , Echo Point , Tata Tea Mus...",35967.0
4,Premium,4N Pattaya . 3N Bangkok,"[Pattaya, Bangkok]",12-12-2021,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,"[Spicejet, Go Air]",0,5,"[ Coral Island Tour with Indian Lunch, Join Sp...",25584.0
...,...,...,...,...,...,...,...,...,...,...,...
20995,Deluxe,2N Shimla . 3N Manali . 1N New Delhi,"[Shimla, Manali, New Delhi]",15-04-2021,Summit Thistle Villa Resort & Spa:Four|Snow Va...,Mumbai,[Not Available],1,3,"[ Mall road , Indira Bunglow , Himalayan Zoo...",16168.5
20996,Luxury,1N Srinagar . 1N Gulmarg . 2N Pahalgam . 3N Sr...,"[Srinagar, Gulmarg, Pahalgam, Srinagar]",30-08-2021,The Orchard Retreat & Spa:4.6|The Khyber Himal...,New Delhi,"[Vistara, Vistara]",0,4,"[ Dal Lake , Gondola Point , Avantipura Ruin...",33770.5
20997,Deluxe,5N Colombo,[Colombo],27-02-2021,Mount Lavinia Hotel-MMT HOLIDAYS SPECIAL:4.0,New Delhi,"[IndiGo, IndiGo, IndiGo, IndiGo]",1,3,"[ Sigiriya Excursion , Nuwara Eliya Excursion ]",29964.0
20998,Deluxe,2N Darjeeling . 2N Pelling . 2N Gangtok,"[Darjeeling, Pelling, Gangtok]",13-01-2022,Ramada by Wyndham Darjeeling Gandhi Road:4.4|T...,New Delhi,[IndiGo],1,3,"[ Tiger Hill , Himalayan Mountaineering Insti...",28145.5


None

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


None

TypeError: 'NoneType' object is not subscriptable

In [ ]:
### Displaying processed data

print('================TRAINING DATA==================')
display(final_train_data)
print(final_train_data.shape)

print('================VALIDATION DATA==================')
display(final_validation_data)
print(final_validation_data.shape)

In [ ]:
### Saving processed data to csv

final_train_data.to_csv('dataset\Train_prepared.csv') 
final_validation_data.to_csv('dataset\Test_prepared.csv')